In [ ]:
# Importing necessary libraries and modules
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
import numpy as np

In [ ]:
# Loading and preprocessing the data
text = open('stories.txt').read().lower()
# Initialize a tokenizer
tokenizer = Tokenizer()
# Fit the tokenizer on text
tokenizer.fit_on_texts([text])
# Calculate the total number of unique words in the text
total_words = len(tokenizer.word_index) + 1

In [ ]:
# Create input sequences using list of tokens
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [ ]:
# Pad sequences for consistent length
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [ ]:
# Creating predictors and label
predictors, label = input_sequences[:,:-1], input_sequences[:,-1]
# Convert label into one-hot encoding
label = tf.keras.utils.to_categorical(label, num_classes=total_words)


In [ ]:

# Building the LSTM model
model = Sequential([
    Embedding(total_words, 200, input_length=max_sequence_len-1),
    Bidirectional(LSTM(200)),
    Dense(total_words, activation='softmax')
])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
history = model.fit(predictors, label, epochs=300, verbose=0)  # Train the model for 100 epochs

In [ ]:
# Function to generate a story based on a given seed text
def story_generator(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')

        predictions = model.predict(token_list)

        predicted = np.argmax(predictions)

        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break

        seed_text += " " + output_word

    return seed_text


In [ ]:
# Generate a story
input_text= "In the hustle and bustle of ipoti"
print(story_generator(input_text, 50, model, max_sequence_len))  # Print the generated story


1/1 [==============================] - 0s 33ms/step
In the hustle and bustle of ipoti the city the friends also learned about the wider context of the ancient world including the people who had lived and worshipped in the area they explored nearby archaeological sites and museums uncovering artifacts and stories that shed light on the lives and beliefs of those who had come before
